In [1]:
import gmsh
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.tri import Triangulation
from matplotlib import cm, patches, collections

In [2]:
experiment='run_330d_iabp'

In [3]:
mesh_file='/bettik/alberta/small-arctic-10km-iabp/experiments/run_330d_iabp/par32small_arctic_10km.msh'

In [4]:
gmsh.initialize()
file = gmsh.open(mesh_file)

Info    : Reading '/bettik/alberta/small-arctic-10km-iabp/experiments/run_330d_iabp/par32small_arctic_10km.msh'...
Info    : 66212 nodes
Info    : 130689 elements
Info    : Converting old partitioning...                                   
Info    : Creating partition topology...
Info    :  - Creating partition curves
Info    :  - Creating partition points
Info    : Done creating partition topology
Info    : Done reading '/bettik/alberta/small-arctic-10km-iabp/experiments/run_330d_iabp/par32small_arctic_10km.msh'


In [5]:
# Number of partitions (i.e. as many partitions as processors on which the simulation has run)
NP =  gmsh.model.getNumberOfPartitions()
print(NP)

# Get all Entities from gmsh file
entities = gmsh.model.getEntities()

32


In [6]:
def GetPartitionsFromEntities(entities,NP):
    """ Gets mesh triangles and nodes coordinates from reading the entities of a gmsh file.
    
    Reads all entities from the gmsh model and look for "Partition surfaces". Loops over the partitions 
    and gets for each partition, all the nodes includes in the partitions (reads their x,y coordinates) 
    and all the mesh elements corresponding to this partitions (i.e. the triangles).
    
    Parameters
    ----------
    entities : Previously read from gmsh file 
        with 'entities = gmsh.model.getEntities()'

    NP : int
        Number of partitions (read from gmsh file with
        'NP=gmsh.model.getNumberOfPartitions()'
        
    Returns
    -------
    3 array_like of size NPx5000
        bigelemtags,bignodes_x,bignodes_y
        containing , for each of the NP partition, the tag of each elements, the x and y coordinates of each node.
    """
    bignodes_x = np.zeros((NP,5000))
    bignodes_y = np.zeros((NP,5000))
    bigelemtags = np.zeros((NP,5000))
    for ipart in range(0,NP):
        icount=-1
        for e in entities:
            if (gmsh.model.getType(e[0], e[1])=="Partition surface"):
                partitions = gmsh.model.getPartitions(e[0], e[1])
                if len(partitions):
                    if ipart in partitions:
                        # get the mesh nodes for each elementary entity
                        nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(e[0], e[1])
                        # Get the mesh elements for the entity (dim, tag):
                        elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(e[0], e[1])
                        # save x,y coordinates of all nodes in the given partition (ipart)
                        bignodes_x[ipart,:len(nodeCoords[0::3])] = nodeCoords[0::3]
                        bignodes_y[ipart,:len(nodeCoords[1::3])] = nodeCoords[1::3]
                        # save the elements tag in the given partition
                        bigelemtags[ipart,:len(elemTags[0])] = elemTags[0]
    return bigelemtags,bignodes_x,bignodes_y


def preparecollection4plt(patchlist,c):
    """ Prepare to plot the patch for each partiton
    
    
    Parameters
    ----------
    patchlist : computed from makecollectionpart

    c : hex color
        color to plot the partition with
        
    Returns
    -------
    pc the patch collection to plot
        

    """
    pc = collections.PatchCollection(patchlist, cmap='binary', alpha=1)
    pc.set_array(np.zeros(1))
    pc.set_edgecolor(c)
    return pc

def makecollectionpart(bigelemtags,ipart,sfac):
    """ Collect all triangles to plot for a given partiton
    
    
    Parameters
    ----------
    bigelemtags : array of all tags of elements corresponding to each of the NP partitons

    ipart : int 
        partition index
    
    sfac  : float
        
        scale factor to convert the coordinates with (for example from m to km)
    Returns
    -------
    patch_list corresponding to the given partition
        
    """

    #Set up the triangles
    patch_list  = []
    rgba_list = []
    nodes1_x = []
    nodes1_y = []
    nodes2_x = []
    nodes2_y = []
    nodes3_x = []
    nodes3_y = []
    part = []
    tag = []

    for t in np.arange(len(bigelemtags[ipart,:])):
        #print(t)
        tag1 = int(bigelemtags[ipart,t])
        if tag1>0:
            eltyp,nodetags,dim,tag2 = gmsh.model.mesh.get_element(tag1)
            part.append(tag2)

            tag.append(t)

            n1=gmsh.model.mesh.get_node(nodetags[0])
            n1_coord=n1[0]
            n1_x=n1_coord[0]*sfac
            n1_y=n1_coord[1]*sfac
            nodes1_x.append(n1_x)
            nodes1_y.append(n1_y)

            n2=gmsh.model.mesh.get_node(nodetags[1])
            n2_coord=n2[0]
            n2_x=n2_coord[0]*sfac
            n2_y=n2_coord[1]*sfac
            nodes2_x.append(n2_x)
            nodes2_y.append(n2_y)

            n3=gmsh.model.mesh.get_node(nodetags[2])
            n3_coord=n3[0]
            n3_x=n3_coord[0]*sfac
            n3_y=n3_coord[1]*sfac
            nodes3_x.append(n3_x)
            nodes3_y.append(n3_y)

            ccl   = []
            ccl.append((n1_x,n1_y))
            ccl.append((n2_x,n2_y))
            ccl.append((n3_x,n3_y))
            ccl.append(ccl[0])
            
            patch_list.append(patches.Polygon(ccl,True,linewidth=0))
    return patch_list        


In [7]:
bigelemtags,bignodes_x,bignodes_y = GetPartitionsFromEntities(entities,NP)               

In [13]:
bignodes_y

array([[      0.        ,       0.        ,       0.        , ...,
              0.        ,       0.        ,       0.        ],
       [ 264884.8770639 ,  270765.24617139,  279453.31133109, ...,
              0.        ,       0.        ,       0.        ],
       [ 128376.72624263,  118047.05986627,  134867.45322203, ...,
              0.        ,       0.        ,       0.        ],
       ...,
       [-376408.29949892, -386196.2896714 , -377734.89809865, ...,
              0.        ,       0.        ,       0.        ],
       [-436501.26305056, -446064.82131929, -443771.0708608 , ...,
              0.        ,       0.        ,       0.        ],
       [-608774.30177583, -615746.26968407, -607958.77528646, ...,
              0.        ,       0.        ,       0.        ]])

In [8]:
# scale factor so that x and y are in km instead of m
sfac=1e-3

# initialize a list that will collect the NP lists corresponding to each of the NP partitions
allpatchlists=[]

# loop on the partitions
for ipart in range(0,NP):
    allpatchlists.append(makecollectionpart(bigelemtags,ipart,sfac))


In [9]:
# create a color_list of NP different colors
cmap = cm.get_cmap('Spectral', NP)
color_list = [matplotlib.colors.rgb2hex(cmap(i)[:3]) for i in range(cmap.N)]


In [15]:
np.savetxt('bigelemtags.txt',bigelemtags)
np.savetxt('bignodes_x.txt',bignodes_x)
np.savetxt('bignodes_y.txt',bignodes_y)


In [ ]:
with open("color.txt", "w") as file:
    file.write(str(color_list))
